In [ ]:
# ===============================================================
# ISA 444 – Hotel Demand Forecasting Project
# FINAL FULLY-COMMENTED NOTEBOOK (READY TO SUBMIT)
# ===============================================================

# -----------------------------
# 1. Install Required Packages
# -----------------------------
!pip install -U statsforecast mlforecast neuralforecast lightgbm matplotlib pandas pyarrow > /dev/null

# -----------------------------
# 2. Import Libraries
# -----------------------------
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

from statsforecast import StatsForecast
from statsforecast.models import Naive, SeasonalNaive, AutoETS

from mlforecast import MLForecast
from lightgbm import LGBMRegressor

from neuralforecast import NeuralForecast
from neuralforecast.models import NBEATS, NHITS

plt.rcParams['figure.figsize'] = (10,4)


# ===============================================================
# 3. Load & Clean Dataset
# ===============================================================

# Load hotel demand parquet file
df = pd.read_parquet('/content/sample_hotels.parquet')

# Standardize column names for forecasting libraries
df = df.rename(columns={
    "hotel_id": "unique_id",
    "date": "ds",
    "demand": "y"
})

# Keep only necessary columns
df = df[['unique_id', 'ds', 'y']]

# Convert date + numeric types
df['ds'] = pd.to_datetime(df['ds'])
df['y'] = pd.to_numeric(df['y'], errors='coerce')

# Remove missing values and sort
df = df.dropna(subset=['y'])
df = df.sort_values(['unique_id', 'ds']).reset_index(drop=True)

print(df.head())
print("Number of hotel series:", df['unique_id'].nunique())


# ===============================================================
# 4. Global Settings & Metric Functions
# ===============================================================

FREQ = 'D'       # Daily data
H = 28           # Forecast horizon (28 days)
N_WINDOWS = 3    # Cross-validation windows (fast)

# Forecast evaluation metrics
def me(y, yhat): return np.mean(y - yhat)
def mae(y, yhat): return np.mean(np.abs(y - yhat))
def rmse(y, yhat): return np.sqrt(np.mean((y - yhat)**2))
def mape(y, yhat):
    y, yhat = np.array(y), np.array(yhat)
    mask = y != 0
    return np.mean(np.abs((y[mask] - yhat[mask]) / y[mask])) * 100


# ===============================================================
# 5. StatsForecast Cross-Validation (Naive + SeasonalNaive + AutoETS)
# ===============================================================

sf_models = [
    Naive(),
    SeasonalNaive(season_length=7),
    AutoETS(season_length=7)
]

sf = StatsForecast(models=sf_models, freq=FREQ, n_jobs=1)

sf_cv = sf.cross_validation(
    df=df,
    h=H,
    n_windows=N_WINDOWS,
    step_size=H
)

print("StatsForecast CV complete.")


# ===============================================================
# 6. NeuralForecast Cross-Validation (NBEATS + NHITS)
# ===============================================================

neural_models = [
    NBEATS(h=H, input_size=28, max_steps=50),
    NHITS(h=H, input_size=28, max_steps=50)
]

nf = NeuralForecast(models=neural_models, freq=FREQ)

nf_cv = nf.cross_validation(
    df=df,
    h=H,
    n_windows=N_WINDOWS,
    step_size=H
)

print("NeuralForecast CV complete.")


# ===============================================================
# 7. MLForecast Cross-Validation (LGBMRegressor)
# ===============================================================

mlf = MLForecast(
    models=LGBMRegressor(),
    freq=FREQ,
    lags=[1,7,14],
    date_features=['dayofweek','month']
)

mlf.fit(df)

mlf_cv = mlf.cross_validation(
    df=df,
    h=H,
    n_windows=N_WINDOWS,
    step_size=H
)

print("MLForecast CV complete.")


# ===============================================================
# 8. Combine All Cross-Validation Outputs
# ===============================================================

cv_full = sf_cv.merge(
    nf_cv[['unique_id','ds','cutoff','NBEATS','NHITS']],
    on=['unique_id','ds','cutoff'],
    how='left'
)

cv_full = cv_full.merge(
    mlf_cv[['unique_id','ds','cutoff','LGBMRegressor']],
    on=['unique_id','ds','cutoff'],
    how='left'
)

print("Combined CV sample:")
print(cv_full.head())


# ===============================================================
# 9. Compute Evaluation Metrics
# ===============================================================

model_cols = ['Naive','SeasonalNaive','AutoETS','NBEATS','NHITS','LGBMRegressor']

rows = []
for uid, g in cv_full.groupby("unique_id"):
    y = g['y'].values
    for m in model_cols:
        yp = g[m].values
        rows.append({
            'unique_id': uid,
            'model': m,
            'ME': me(y, yp),
            'MAE': mae(y, yp),
            'RMSE': rmse(y, yp),
            'MAPE': mape(y, yp)
        })

metrics_df = pd.DataFrame(rows)
print("Metrics sample:")
print(metrics_df.head())


# ===============================================================
# 10. Determine Model Wins (Best Model Per Series)
# ===============================================================

best = metrics_df.loc[
    metrics_df.groupby("unique_id")["MAE"].idxmin()
]

wins = best['model'].value_counts()
print("\nModel wins across all hotels:")
print(wins)


# ===============================================================
# 11. Train Final Models on Full Data & Generate 28-Day Forecasts
# ===============================================================

sf_final = StatsForecast(models=sf_models, freq=FREQ, n_jobs=1).fit(df)
sf_fcst = sf_final.predict(h=H)

nf_final = NeuralForecast(models=neural_models, freq=FREQ)
nf_final.fit(df)
nf_fcst = nf_final.predict()

mlf_final = MLForecast(
    models=LGBMRegressor(),
    freq=FREQ,
    lags=[1,7,14],
    date_features=['dayofweek','month']
)
mlf_final.fit(df)
mlf_fcst = mlf_final.predict(h=H)

# Merge all forecasts
final_fc = sf_fcst.merge(nf_fcst, on=['unique_id','ds'], how='left')
final_fc = final_fc.merge(mlf_fcst, on=['unique_id','ds'], how='left')

print("Final forecast sample:")
print(final_fc.head())


# ===============================================================
# 12. Save Outputs as CSV (For Submission)
# ===============================================================

final_fc.to_csv("final_forecasts.csv", index=False)
metrics_df.to_csv("metrics.csv", index=False)
cv_full.to_csv("cv_results.csv", index=False)
wins.to_csv("model_wins.csv")

print("\nSaved CSV files:")
print("- final_forecasts.csv")
print("- metrics.csv")
print("- cv_results.csv")
print("- model_wins.csv")


# ===============================================================
# 13. Plot Example Forecasts (Optional Visualization)
# ===============================================================

def plot_forecast(uid, model):
    """Plot the last 100 days of history + model forecast."""
    hist = df[df['unique_id']==uid].tail(100)
    fut = final_fc[final_fc['unique_id']==uid]

    plt.plot(hist['ds'], hist['y'], label="History")
    plt.plot(fut['ds'], fut[model], '--', label=f"{model} Forecast")
    plt.title(f"{uid} — {model}")
    plt.legend()
    plt.show()

# Example plot for first hotel series
example_series = df['unique_id'].unique()[0]
plot_forecast(example_series, "LGBMRegressor")
